In [2]:
import pandas as pd
from pymongo import MongoClient
import numpy as np
import scipy.sparse as  sps
import scipy.io as io
# import blogpost_data_source

In [3]:
host = 'localhost'
port = '27017'
db = 'alizeh'
client = MongoClient('mongodb://localhost:27017/')
dbCursor = client['alizeh']
mediumClient = client['medium']
cursor = dbCursor['user_activity'].find({},{'blogpost_id':1, 'rating':1, 'user_id':1,'_id':0})
df =  pd.DataFrame(list(cursor))


# db = client.database_name
# collection = db.collection_name
# data = pd.DataFrame(list(collection.find()))

In [4]:
# pd.isna(df['rating'])
# df[df['visited_post'][] != 'NaN']

In [5]:
extracted_df = df[['blogpost_id', 'rating', 'user_id']].copy()
extracted_df.dropna(axis=0, inplace=True)
print(extracted_df.shape)
print(extracted_df.head())

(9253, 3)
    blogpost_id  rating                   user_id
0  140b4bf94d9f     2.0  5acb783dab23ba134d0580c8
1  33a063e8b147     5.0  5acb783dab23ba134d0580c8
2  f74463ed54ee     5.0  5acb783dab23ba134d0580c8
3  aa8325c6145f     1.0  5acb783dab23ba134d0580c8
4  bd4b418f3cfb     5.0  5acb783dab23ba134d0580c8


In [6]:
# extracted_df.set_index(['user_id', 'blogpost_id'], inplace=True)
# extracted_df
# extracted_df[extracted_df.index.labels[1] < 0]

In [7]:
# spsMat = sps.coo_matrix((extracted_df.rating,((extracted_df.index.labels[0], extracted_df.index.labels[1]))))
# spsMat.todense()
# io.mmwrite('test123',spsMat)
# max(extracted_df.index.labels[0]) #user_id
# max(extracted_df.index.labels[1]) #blog_id

In [8]:
# def generateBlogIdXTags():
#     for _ in extracted_df.index.levels[0]:
mediumCursor = mediumClient['post'].aggregate([{
    '$project': {'_id':0, 'blogpost_id':'$value.id', 'tags':'$value.virtuals.tags.slug'}
    }])

df_post_x_tags =  pd.DataFrame(list(mediumCursor))
# spsTags = sps.coo_matrix((temp.one,(temp.index.labels[0], temp.index.labels[1])))
# generateBlogIdXTags()

In [9]:
df_post_x_tags.head()
df_post_x_tags.dropna(axis=0, inplace=True)
df_post_x_tags.shape
df_post_x_tags.head()

,blogpost_id,tags
0,d8833b94bc05,"[life-hacking, time-tracking, lifelogging, lif..."
1,6558eedcf0c1,[]
3,93ebd727e182,"[lean-startup, startup, startup-lessons, mvp, ..."
5,9862438d4720,[]
6,a2e17b407e1d,"[tech, accelerator, techstars, incubator, know..."


In [10]:
extracted_df.head()

,blogpost_id,rating,user_id
0,140b4bf94d9f,2.0,5acb783dab23ba134d0580c8
1,33a063e8b147,5.0,5acb783dab23ba134d0580c8
2,f74463ed54ee,5.0,5acb783dab23ba134d0580c8
3,aa8325c6145f,1.0,5acb783dab23ba134d0580c8
4,bd4b418f3cfb,5.0,5acb783dab23ba134d0580c8


In [11]:
# temprows = []
# extracted_df.apply(lambda row:[temprows.append([str(row['blogpost_id']), row['rating'], str(row['user_id'])])], axis=1)
# extracted_df = pd.DataFrame(temprows, columns=extracted_df.columns)
# extracted_df.head()
# temprows = []
# df_post_x_tags.apply(lambda row:[temprows.append([str(row['blogpost_id']), row['tags']])], axis=1)
# df_post_x_tags = pd.DataFrame(temprows, columns=df_post_x_tags.columns)
# df_post_x_tags.head()

In [12]:
extracted_df['matches'] =  extracted_df['blogpost_id'].isin(df_post_x_tags['blogpost_id'])
extracted_df['matches'].unique()
extracted_df.drop(['matches'], axis=1, inplace=True)

In [13]:
joined = df_post_x_tags.merge(extracted_df,left_on='blogpost_id', right_on='blogpost_id',how='left')
# extracted_df.set_index('blogpost_id', 'user_id').join(df_post_x_tags.set_index('blogpost_id'))
print(joined.shape)
joined.dropna(axis=0, inplace=True)
joined.head()

(18241, 4)


,blogpost_id,tags,rating,user_id
14,1fe04ad83f6e,"[react, rails, browserify, integrate-reactjs, ...",3.0,5acb783eab23ba134d05829e
21,ab448906c4e1,"[javascript, es6, web-development]",3.0,5acb783dab23ba134d0580ca
22,ab448906c4e1,"[javascript, es6, web-development]",5.0,5acb783eab23ba134d05811d
23,e2ba3920f313,"[javascript, es6, ruby]",5.0,5acb783eab23ba134d05821b
27,52419f47d2a3,"[javascript, nodejs, frp, rxjs]",3.0,5acb783dab23ba134d0580f4


In [14]:

interaction = joined[['blogpost_id', 'user_id', 'rating']].copy()
blog_feature = joined[['blogpost_id', 'tags']].copy()
interaction['user_id'].unique().size

interaction.drop_duplicates(subset=['blogpost_id','user_id'],keep='first')
interaction.set_index(['user_id', 'blogpost_id'], inplace=True)
interaction.index.levels[1].size
interaction.head()

,,rating
user_id,blogpost_id,
5acb783eab23ba134d05829e,1fe04ad83f6e,3.0
5acb783dab23ba134d0580ca,ab448906c4e1,3.0
5acb783eab23ba134d05811d,ab448906c4e1,5.0
5acb783eab23ba134d05821b,e2ba3920f313,5.0
5acb783dab23ba134d0580f4,52419f47d2a3,3.0


In [15]:
blog_feature
#need to unwind the tags
temprows = []
blog_feature.apply(lambda row:[temprows.append([row['blogpost_id'], eachTag]) for eachTag in row.tags], axis=1)
blog_feature = pd.DataFrame(temprows, columns=blog_feature.columns)
blog_feature['tags'].unique().size

1844

In [16]:
blog_feature['match'] = 1
blog_feature.set_index(['blogpost_id','tags'], inplace=True)
blog_feature.index.levels[1]
# sps.csr_matrix((blog_feature['match'],(blog_feature.index.labels[0], blog_feature.index.labels[1]))).shape
# scrBlogFeature = sps.csr_matrix(blog_feature.values)
# scrBlogFeature
blog_feature.head()

match
blogpost_id  tags                    
1fe04ad83f6e react                  1
             rails                  1
             browserify             1
             integrate-reactjs      1
             rails-application      1

In [17]:
blog_feature.shape

(43857, 1)

In [18]:
spsInteraction = sps.coo_matrix((interaction.rating,((interaction.index.labels[0], interaction.index.labels[1]))))
spsInteraction.shape

(478, 1844)

In [19]:
scrBlogTags = sps.csr_matrix((blog_feature['match'],(blog_feature.index.labels[0], blog_feature.index.labels[1])))
tags = blog_feature.index.levels[1]

In [20]:
scrBlogTags.shape

(1844, 1844)

In [21]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(spsInteraction,test_size=0.25)
train.shape

(358, 1844)

In [22]:
from lightfm import LightFM
model = LightFM(loss='warp')
%time model.fit(spsInteraction,item_features=scrBlogTags, epochs=15, num_threads=3)

CPU times: user 514 ms, sys: 659 µs, total: 514 ms
Wall time: 231 ms


In [23]:
type(model.item_embeddings)

numpy.ndarray

In [24]:
np.array(interaction.index.levels[1].size)

array(1844)

In [25]:
scores = model.predict(0, np.arange(interaction.index.levels[1].size),item_features=scrBlogTags) # user_id, 
top_items = tags[np.argsort(-scores)]
top_items_score = scores[np.argsort(-scores)]
print(top_items[0:5])

Index([u'cameras', u'version-control', u'sql-injection', u'amazon-go',
       u'miami'],
      dtype='object', name=u'tags')


In [26]:
pd.DataFrame(data={'top_items': top_items[0:5], 'score': top_items_score[0:5]})

,score,top_items
0,4.003980e+12,cameras
1,3.434297e+11,version-control
2,3.891410e+10,sql-injection
3,2.659171e+10,amazon-go
4,2.655106e+10,miami


In [27]:
%time blogs_user_wud_like = interaction.index.levels[1][np.argsort(-scrBlogTags.dot(top_items_score))]

CPU times: user 894 µs, sys: 0 ns, total: 894 µs
Wall time: 594 µs


In [28]:
blogs_user_wud_like[0:5]

Index([u'b0051fbd5622', u'86c7ced839a5', u'16d9eb86309c', u'b7a5a5549655',
       u'e8ad2eb406c5'],
      dtype='object', name=u'blogpost_id')